In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
import mysql.connector
import pickle
import scipy.stats as st

%matplotlib inline

In [2]:
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


We define interaction as: activity or engagement between the client and the step in the process being analyzed per visit. 
So attempting step 3 4 times is one interaction.

We define error as an interaction with a step attempt higher than 1,indicating possible confusion or errors. So attempting step 3 4 times is considered an error.

We define non-error as an interaction with a step attempt equal to 1. So attempting step 3 one time is considered non-error, 
even if they abandon the process at that step.

In [3]:
# Connect to MySQL database
conn = mysql.connector.connect(host='127.0.0.1', user='root', passwd=password)
cursor = conn.cursor()

#Select the database
cursor.execute("USE vanguard")

In [6]:
cursor.execute("""
                  SELECT 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        COUNT(interactions.process_step) AS step_attempt,
                        group_id.variation
                    FROM clients
                    JOIN interactions ON clients.client_id = interactions.client_id
                    JOIN group_id ON group_id.client_id = clients.client_id
                    WHERE group_id.variation IN ('Control', 'Test')
                    GROUP BY 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        group_id.variation;
             """)

results = cursor.fetchall()

total_interactions = pd.DataFrame(results)

In [27]:
"""
total interactions with the website by client, per visit, per step from people in test or control. 
total interactions with the site: 239331
"""

total_interactions.rename(columns = {0:'client_id', 1:'visit_id', 2:'process_step', 3:'step_attempt', 4:'group_id'}, inplace=True)
len(total_interactions)

"""
If we want the error rate, we filter out any times the step is only done once to focus on errors.
interactions with error = 50650 
"""

error_filter = total_interactions[total_interactions["step_attempt"] > 1]
len(error_filter)

"""So if we know the number of total attempts and the number of error attempts, we can calculate an error rate"""

error_rate_overall = round(len(error_filter)/len(total_interactions)*100,2)
print(f"The overall error rate is: {error_rate_overall}%") 

The overall error rate is: 21.16%


In [23]:
#we look at interactions from the test group
cursor.execute("""
                  SELECT 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        COUNT(interactions.process_step) AS step_attempt,
                        group_id.variation
                    FROM clients
                    JOIN interactions ON clients.client_id = interactions.client_id
                    JOIN group_id ON group_id.client_id = clients.client_id
                    WHERE group_id.variation IN ('Test')
                    GROUP BY 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        group_id.variation;
             """)

results = cursor.fetchall()

test_group_interactions = pd.DataFrame(results)


The error rate for the test group is: 22.46%


In [22]:
#the test group had 130159 interactions overall

test_group_interactions.rename(columns = {0:'client_id', 1:'visit_id', 2:'process_step', 3:'step_attempt', 4:'group_id'}, inplace=True)
test_group_interactions

#the test group had 29239 attempts with errors

test_error_filter = test_group_interactions[test_group_interactions["step_attempt"] > 1]
test_error_filter

test_error_rate = round(len(test_error_filter)/len(test_group_interactions)*100,2)
print(f"The error rate for the test group is: {test_error_rate}%") 

The error rate for the test group is: 22.46%


In [24]:
#we look at interactions from the control group
cursor.execute("""
                  SELECT 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        COUNT(interactions.process_step) AS step_attempt,
                        group_id.variation
                    FROM clients
                    JOIN interactions ON clients.client_id = interactions.client_id
                    JOIN group_id ON group_id.client_id = clients.client_id
                    WHERE group_id.variation IN ('Control')
                    GROUP BY 
                        clients.client_id,
                        interactions.visit_id,
                        interactions.process_step,
                        group_id.variation;
             """)

results = cursor.fetchall()

control_group_interactions = pd.DataFrame(results)


In [30]:
#the control group had 109172 interactions overall

control_group_interactions.rename(columns = {0:'client_id', 1:'visit_id', 2:'process_step', 3:'step_attempt', 4:'group_id'}, inplace=True)
len(control_group_interactions)

#the control group had 21411 interactions with errors

control_error_filter = control_group_interactions[control_group_interactions["step_attempt"] > 1]
len(control_error_filter)

control_error_rate = round(len(control_error_filter)/len(control_group_interactions)*100,2)
print(f"The error rate for the control group is: {control_error_rate}%") 

The error rate for the control group is: 19.61%


In [44]:
#what is the statistical significance of the error rates

from scipy.stats import chi2_contingency

# Error rates
error_rate_test = 0.2246  # 22.46%
error_rate_control = 0.1961  # 19.61%

# Sample sizes (you need to provide these)
n_test =  len(test_group_interactions)
n_control =  len(control_group_interactions)

# Number of errors and non-errors in each group
test_errors = error_rate_test * n_test
test_non_errors = n_test - test_errors

control_errors = error_rate_control * n_control
control_non_errors = n_control - control_errors

# Create a contingency table
observed = np.array([[test_errors, control_errors],
                     [test_non_errors, control_non_errors]])

# Perform chi-square test
chi2, p_value, _, _ = chi2_contingency(observed)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

Chi-square statistic: 288.9068783607042
p-value: 8.604767700438106e-65


In [50]:
er_control = total_interactions[total_interactions["group_id"] == "Control"]["step_attempt"]
er_test = total_interactions[total_interactions["group_id"] == "Test"]["step_attempt"]

t_statistic, p_value = st.ttest_ind(er_control, er_test, equal_var=False)

print("T-test result:")
print(f"T-statistic: {t_statistic:.6f}")
print(f"P-value: {p_value:.6e}")

T-test result:
T-statistic: -14.459045
P-value: 2.303818e-47


In [55]:
#we look at only the interactions with errors of clients who reached the confirm stage
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Control', 'Test')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation
HAVING COUNT(*) > 1;
             """)

results = cursor.fetchall()

confirmed_only_errors = pd.DataFrame(results)

In [54]:
#we look at all the interactions of clients who reached the confirm stage
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Control', 'Test')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation;
             """)

results = cursor.fetchall()

confirmed_with_errors = pd.DataFrame(results)

In [57]:
error_rate_confirmers = round(len(confirmed_only_errors)/len(confirmed_with_errors)*100,2)
print(f"The overall error rate of people who completed the process is: {error_rate_confirmers }%") 

The overall error rate of people who completed the process is: 17.53%


In [60]:
#we look at only the interactions with errors of clients who reached the confirm stage in the test group
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Test')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation
HAVING COUNT(*) > 1;
             """)

results = cursor.fetchall()

errors_finished_test = pd.DataFrame(results)

len(errors_finished_test)

19271

In [61]:
#we look at only the interactions with errors of clients who reached the confirm stage in the test group
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Test')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation;
             """)

results = cursor.fetchall()

all_finished_test = pd.DataFrame(results)

len(all_finished_test)

105592

In [73]:
error_rate_confirmers_in_test = round(len(errors_finished_test)/len(all_finished_test)*100,2)
error_rate_confirmers_in_test

13.77

In [69]:
#we look at only the interactions with errors of clients who reached the confirm stage in the control group
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Control')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation
HAVING COUNT(*) > 1;
             """)

results = cursor.fetchall()

errors_finished_control = pd.DataFrame(results)

len(errors_finished_control)

14545

In [70]:
#we look at only the interactions with errors of clients who reached the confirm stage in the control group
cursor.execute("""
                  SELECT 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation,
    COUNT(*) AS attempts
FROM clients
JOIN interactions ON clients.client_id = interactions.client_id
JOIN group_id ON group_id.client_id = clients.client_id
WHERE group_id.variation IN ('Control')
 AND clients.client_id IN (
        SELECT DISTINCT clients.client_id 
        FROM clients
        JOIN interactions ON clients.client_id = interactions.client_id
        WHERE interactions.process_step = '4')
GROUP BY 
    clients.client_id,
    interactions.visit_id,
    interactions.process_step,
    group_id.variation;
             """)

results = cursor.fetchall()

all_finished_control = pd.DataFrame(results)

len(all_finished_control)

87324

In [72]:
error_rate_confirmers_in_control = round(len(errors_finished_control)/len(all_finished_control)*100,2)
error_rate_confirmers_in_control

16.66

# Conclusions

We define interaction as: activity or engagement between the client and the step in the process being analyzed. 
So attempting step 3 4 times is one interaction.

We define error as an interaction with a step attempt higher than 1,indicating possible confusion or errors. So attempting step 3 4 times is considered an error.

We define non-error as an interaction with a step attempt equal to 1. So attempting step 3 one time is considered non-error, 
even if they abandon the process at that step.


## chi-square
The null hypothesis (H0):there is NO difference in the proportions of errors between the two groups

Alternative hypothesis (H1):there IS a difference in the proportions of errors between the two groups

## t-test
The null hypothesis (H0): any observed difference in error rates between the control and test groups is attributed to random chance or sampling variability

Alternative hypothesis (H1): any observed difference in error rates between the control and test groups is NOT attributed to random chance or sampling variability

α = 0.05

In [75]:
print(f"The error rate for the test group is: {test_error_rate}%") 
print(f"The error rate for the control group is: {control_error_rate}%") 


print(f"The error rate of all completed processes is: {error_rate_confirmers}%") 
print(f"The error rate for the test group that completed the process is: {error_rate_confirmers_in_test}%") 
print(f"The error rate for the control group that completed the process is: {error_rate_confirmers_in_control}%") 

The error rate for the test group is: 22.46%
The error rate for the control group is: 19.61%
The error rate of all completed processes is: 17.53%
The error rate for the test group that completed the process is: 13.77%
The error rate for the control group that completed the process is: 16.66%


The control group had a lower error rate than the test group, meaning the control group was less likely to visit a step more than once. This could be due to previous familiarity with the process, where in the test group the process has changes to what they're used to.

In [51]:
print("Chi-square statistic:", chi2)
print("p-value:", p_value)


print("T-test result:")
print(f"T-statistic: {t_statistic:.6f}")
print(f"P-value: {p_value:.6e}")

Chi-square statistic: 288.9068783607042
p-value: 2.303817813154474e-47
T-test result:
T-statistic: -14.459045
P-value: 2.303818e-47


For chi-test we can reject the null hypothesis. This suggests that there is a statistically significant difference in the proportions of errors between the control and test groups.

For t-test: we can reject the null hypothesis. This suggests any observed difference in error rates between the control and test groups is NOT attributed to random chance or sampling variability.